# Main NoteBook - Random Walk + AprilTag Detection + Action

In [1]:
# Imports
import time
import sys
import math
from dataclasses import dataclass
from random import randint
import asyncio

from IPython.display import clear_output
import cv2                                # state of the art computer vision algorithms library
import numpy as np                        # fundamental package for scientific computing
import matplotlib.pyplot as plt           # 2D plotting library producing publication quality figures
import pyrealsense2 as rs                 # Intel RealSense cross-platform open-source API
from pyapriltags import Detector


from unitree_sdk2py.core.channel import ChannelSubscriber, ChannelFactoryInitialize
from unitree_sdk2py.idl.default import unitree_go_msg_dds__SportModeState_
from unitree_sdk2py.idl.unitree_go.msg.dds_ import SportModeState_
from unitree_sdk2py.go2.video.video_client import VideoClient
from unitree_sdk2py.go2.sport.sport_client import (
    SportClient,
    PathPoint,
    SPORT_PATH_POINT_SIZE,
)
from unitree_sdk2py.go2.obstacles_avoid.obstacles_avoid_client import ObstaclesAvoidClient

%matplotlib inline

In [2]:
# setup for add on camera
ChannelFactoryInitialize(0, "eth0")
# ChannelFactoryInitialize()

sport_client = SportClient()  
sport_client.SetTimeout(10.0)
sport_client.Init()

obsclient = ObstaclesAvoidClient()
obsclient.SetTimeout(3.0)
obsclient.Init()

# setup for default camera
client = VideoClient()
client.SetTimeout(1.0)
client.Init()


In [3]:
at_detector = Detector(families='tag36h11',
                       nthreads=1,
                       quad_decimate=1.0,
                       quad_sigma=0.0,
                       refine_edges=1,
                       decode_sharpening=0.25,
                       debug=0)

In [4]:
def move_randomly(vx=0.3, vy=0.3, vyaw=0.3):
    obsclient.UseRemoteCommandFromApi(True)
    obsclient.Move(vx, vy, vyaw)
    time.sleep(1)

def stop_move():
    obsclient.Move(0.0, 0, 0)
    obsclient.UseRemoteCommandFromApi(False)
    time.sleep(1)

def act(code):
    stop_move()
    sport_client.BalanceStand()
    time.sleep(1)
    if code == 0:
        # hello
        sport_client.Hello()
    elif code == 1:
        sport_client.Heart()
    elif code == 2:
        sport_client.Stretch()
        time.sleep(2)
    elif code == 3:
        sport_client.WiggleHips()
        time.sleep(5)
    elif code == 4:
        sport_client.StandDown()
        time.sleep(2)
        sport_client.StandUp()
        time.sleep(2)
    else:
        return
   
    #end
    time.sleep(1)
    sport_client.BalanceStand()
    time.sleep(1)
    move_randomly()
    time.sleep(3)
  

In [5]:
# camera Setup:
pipe = rs.pipeline()
cfg = rs.config()
profile = pipe.start(cfg)

In [6]:
MIN_DISTANCE = 400

# def get 

def go_back():
    while dist < MIN_DISTANCE:
        obsclient.UseRemoteCommandFromApi(True)
        obsclient.Move(-0.1, 0, 0)
        distance = get_distance()
        
    return

In [7]:
def detect_tag_from_main_camera():
    # feed from default camera
    code,data = client.GetImageSample()
    img_1 = np.frombuffer(bytes(data), dtype=np.uint8)
    image_1 = cv2.imdecode(img_1, cv2.IMREAD_COLOR)
    image_1_gray = cv2.cvtColor(image_1, cv2.COLOR_BGR2GRAY)
    clear_output(wait=True)
    # plt.imshow(image_1_gray, cmap="gray")
    # plt.show()
    tags = at_detector.detect(image_1_gray)
    return tags
    # return [t.tag_id for t in tags]
    

def detect_tag():
    frameset = pipe.wait_for_frames()
    color_frame = frameset.get_color_frame()
    color_rgb = np.asanyarray(color_frame.get_data())
    center_crop = color_rgb[:,240:-240,:]
    center_crop_gray = cv2.cvtColor(center_crop, cv2.COLOR_RGB2GRAY)
    tags = at_detector.detect(center_crop_gray)
    return [t.tag_id for t in tags]

In [ ]:
 while True:
     
    # r = (np.random.random(3)-0.5)*0.8
    # move_randomly(*r)
    tags = detect_tag()
    tags_2 = detect_tag_from_main_camera()
    if len(tags_2) > 0:
        print(len(sorted(tags_2)))
        # break
    
    if len(tags) > 0:
        act(tags[0])

In [18]:
stop_move()

In [8]:
def detect_black_block_bottom(image):
    height, width = image.shape[:2]
    
    # # Focus only on the bottom part (e.g., last 100 pixels)
    # roi_height = 100  # Adjustable based on expected block size
    # roi = image[height - roi_height:height, :]
    
    # Convert to grayscale and apply threshold
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    _, thresh = cv2.threshold(gray, 50, 50, cv2.THRESH_BINARY_INV)  # Detect dark regions
    
    # Find contours
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contours = [c for c in contours if cv2.boundingRect(c)[2] > 0.2*width and cv2.boundingRect(c)[1] > 0.7*height and  cv2.boundingRect(c)[3] > 100]
    cv2.drawContours(image, contours, -1, (0, 255, 0), 2)
    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)

        # Check if the detected block spans most of the width (e.g., 80% or more)
        if w >= 0.5 * width and x > gray.shape[0]//2:
            cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
            cv2.putText(image, "Black Block Detected", (50, height - 150),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 3)
            print("Black block detected at the bottom!")
            break

    return image, contours

In [15]:
move_randomly(0.1, 0, 0)
while True:
    # feed from default camera
    code,data = client.GetImageSample()
    img_1 = np.frombuffer(bytes(data), dtype=np.uint8)
    image_1 = cv2.imdecode(img_1, cv2.IMREAD_COLOR)
    image_1_rgb = cv2.cvtColor(image_1, cv2.COLOR_BGR2RGB)
    
    image_1_gray = gray = cv2.cvtColor(image_1, cv2.COLOR_BGR2GRAY)
    _, thresh = cv2.threshold(gray, 50, 255, cv2.THRESH_BINARY_INV)  # Invert for black detection
    
    
    # Get frame dimensions
    height, width, _ = image_1.shape
    
    # Define white rectangle (200 pixels from each side)
    margin = 200
    rect_x1, rect_y1 = margin, 400
    rect_x2, rect_y2 = width - margin, height

    
    detected_image, contours = detect_black_block_bottom(image_1)
    
    cv2.rectangle(image_1, (rect_x1, rect_y1), (rect_x2, rect_y2), (255, 255, 255), 2)
    
    # Check if any edge of the object is outside the rectangle
    if contours:
        print(len(contours))
        max_contour = max(contours, key=cv2.contourArea, default=None)
        x, y, w, h = cv2.boundingRect(max_contour)
        # if x < rect_x1 or (x + w) > rect_x2 or y < rect_y1 or (y + h) > rect_y2: # if condition for entire object to be present inside
        if y > rect_y2:
            cv2.putText(detected_image, "Object Outside!", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 3, (255, 255, 255), 3)
            print("Warning: Object is outside the safe area!")
            # move_randomly(-0.1, 0, 0)
            # time.sleep(1)
        else:
            cv2.putText(detected_image, "Object Inside!", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 3, (255, 255, 255), 3)
            print("Object inside")
            move_randomly(0.1, 0, 0)
    else:
        move_randomly(-0.1, 0, 0)
        time.sleep(1)
    clear_output(wait=True)
    fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(20,12))
    ax[0].imshow(image_1_rgb)
    ax[1].imshow(detected_image)
    plt.show()
    # time.sleep(2)

NameError: name 'detect_black_block_bottom' is not defined

In [16]:
stop_move()

# Using AprilTag

In [17]:

move_randomly()
threshold = 850
while True:
    # # feed from default camera
    # code,data = client.GetImageSample()
    # img_1 = np.frombuffer(bytes(data), dtype=np.uint8)
    # image_1 = cv2.imdecode(img_1, cv2.IMREAD_COLOR)
    # image_1_rgb = cv2.cvtColor(image_1, cv2.COLOR_BGR2RGB)
    
        
    tags_2 = detect_tag_from_main_camera()        
    if tags_2:
        max_bottom_left = 1080
        found = False
        for tag in tags_2:
            if tag.center[1] < threshold:
                found = True
                print("found tag inside the box")
                move_randomly()
                break
        if not found:
            print("no tag found")
            move_randomly(-0.1, 0, 0)
            time.sleep(1)
            move_randomly()
    else:
        move_randomly(-0.1, 0, 0)
        time.sleep(1)
        move_randomly()
        
    r = (np.random.random(3)-0.5)*0.8
    move_randomly(*r)
    
    tags = detect_tag()
    if len(tags_2) > 0:
        print(sorted(tags_2))
        # break
    
    if len(tags) > 0:
        act(tags[0])

KeyboardInterrupt: 

In [16]:
stop_move()